In [1]:
mname = 'irv2_b8'
seed = 723
fold = 2

In [2]:
gpu_id = 0
nfold = 4

In [3]:
# initialize weights from this model
mname0 = 'irv2_b7'

In [4]:
import socket
import timeit
import time
from datetime import datetime
import os
import glob
from collections import OrderedDict
import numpy as np
import pandas as pd
import pickle
import gc
import cv2
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")
import random
import PIL
import pathlib
import math

import torch
from torch.autograd import Variable
import torch.optim as optim
from torch.utils import data
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.utils import make_grid
from torch import nn
from torch.nn import functional as F
from torch.optim.lr_scheduler import LambdaLR, ReduceLROnPlateau, StepLR
from torch.utils.data.sampler import WeightedRandomSampler
import torchvision

from skimage.exposure import histogram, equalize_hist, equalize_adapthist
from skimage.morphology import binary_dilation

import pretrainedmodels
from xception import xception

from tensorboardX import SummaryWriter

from scipy.special import logit
from scipy.ndimage.filters import gaussian_filter

from sklearn.metrics import jaccard_similarity_score, f1_score
from sklearn.preprocessing import MultiLabelBinarizer

import imgaug as ia
from imgaug import augmenters as iaa
import multiprocessing
import threading

from dataloaders import utils
from dataloaders import custom_transforms as tr

# from losses import CombinedLoss, BCELoss2d
from losses import FocalLoss, ThreeWayLoss
import lovasz_losses as L

In [5]:
ori_size = 512
up_size = 512
image_size = 512
final_size = 512

interp = cv2.INTER_AREA
# methods=[("area", cv2.INTER_AREA), 
#          ("nearest", cv2.INTER_NEAREST), 
#          ("linear", cv2.INTER_LINEAR), 
#          ("cubic", cv2.INTER_CUBIC), 
#          ("lanczos4", cv2.INTER_LANCZOS4)]

y_pad = image_size - up_size
y_min_pad = int(y_pad / 2)
y_max_pad = y_pad - y_min_pad

x_pad = image_size - up_size
x_min_pad = int(x_pad / 2)
x_max_pad = x_pad - x_min_pad

print(ori_size, up_size, image_size, final_size)

512 512 512 512


In [6]:
PATH = './'

PATH_TO_TRAIN = PATH + 'train/'
PATH_TO_TEST = PATH + 'test/'
PATH_TO_TARGET = PATH + 'train.csv'
PATH_TO_SUB = PATH + 'sample_submission.csv'

clusters = pd.read_csv('cluster4_folds.csv')
folds = dict(zip(clusters.Id,clusters.cluster4))

LABEL_MAP = {
0: "Nucleoplasm" ,
1: "Nuclear membrane"   ,
2: "Nucleoli"   ,
3: "Nucleoli fibrillar center",   
4: "Nuclear speckles"   ,
5: "Nuclear bodies"   ,
6: "Endoplasmic reticulum"   ,
7: "Golgi apparatus"  ,
8: "Peroxisomes"   ,
9:  "Endosomes"   ,
10: "Lysosomes"   ,
11: "Intermediate filaments"  , 
12: "Actin filaments"   ,
13: "Focal adhesion sites"  ,
14: "Microtubules"   ,
15: "Microtubule ends"   ,
16: "Cytokinetic bridge"   ,
17: "Mitotic spindle"  ,
18: "Microtubule organizing center",  
19: "Centrosome",
20: "Lipid droplets"   ,
21: "Plasma membrane"  ,
22: "Cell junctions"   ,
23: "Mitochondria"   ,
24: "Aggresome"   ,
25: "Cytosol" ,
26: "Cytoplasmic bodies",
27: "Rods & rings"}

In [7]:
print(pretrainedmodels.model_names)

['fbresnet152', 'bninception', 'resnext101_32x4d', 'resnext101_64x4d', 'inceptionv4', 'inceptionresnetv2', 'alexnet', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'inceptionv3', 'squeezenet1_0', 'squeezenet1_1', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19_bn', 'vgg19', 'nasnetamobile', 'nasnetalarge', 'dpn68', 'dpn68b', 'dpn92', 'dpn98', 'dpn131', 'dpn107', 'xception', 'senet154', 'se_resnet50', 'se_resnet101', 'se_resnet152', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'cafferesnet101', 'pnasnet5large', 'polynet']


In [8]:
print(pretrainedmodels.pretrained_settings['dpn92'])

{'imagenet+5k': {'url': 'http://data.lip6.fr/cadene/pretrainedmodels/dpn92_extra-b040e4a9b.pth', 'input_space': 'RGB', 'input_size': [3, 224, 224], 'input_range': [0, 1], 'mean': [0.48627450980392156, 0.4588235294117647, 0.40784313725490196], 'std': [0.23482446870963955, 0.23482446870963955, 0.23482446870963955], 'num_classes': 1000}}


In [9]:
df = pd.read_csv(PATH_TO_TARGET)
df.set_index('Id',inplace=True)
print(df.head())
print(df.shape)

                                       Target
Id                                           
00070df0-bbc3-11e8-b2bc-ac1f6b6435d0     16 0
000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0  7 1 2 0
000a9596-bbc4-11e8-b2bc-ac1f6b6435d0        5
000c99ba-bba4-11e8-b2b9-ac1f6b6435d0        1
001838f8-bbca-11e8-b2bc-ac1f6b6435d0       18
(31072, 1)


In [10]:
file_list = list(df.index.values)

ss = pd.read_csv(PATH_TO_SUB)
ss.set_index('Id',inplace=True)
print(ss.head())
print(ss.shape)

                                      Predicted
Id                                             
00008af0-bad0-11e8-b2b8-ac1f6b6435d0          0
0000a892-bacf-11e8-b2b8-ac1f6b6435d0          0
0006faa6-bac7-11e8-b2b7-ac1f6b6435d0          0
0008baca-bad7-11e8-b2b9-ac1f6b6435d0          0
000cce7e-bad4-11e8-b2b8-ac1f6b6435d0          0
(11702, 1)


In [11]:
test_file_list = list(ss.index.values)
print(file_list[:3], PATH_TO_TRAIN, len(file_list))
print(test_file_list[:3], PATH_TO_TEST, len(test_file_list))

['00070df0-bbc3-11e8-b2bc-ac1f6b6435d0', '000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0', '000a9596-bbc4-11e8-b2bc-ac1f6b6435d0'] ./train/ 31072
['00008af0-bad0-11e8-b2b8-ac1f6b6435d0', '0000a892-bacf-11e8-b2b8-ac1f6b6435d0', '0006faa6-bac7-11e8-b2b7-ac1f6b6435d0'] ./test/ 11702


In [12]:
def image_histogram_equalization(image, number_bins=256):
    # from http://www.janeriksolem.net/2009/06/histogram-equalization-with-python-and.html

    # get image histogram
    image_histogram, bins = np.histogram(image.flatten(), number_bins, density=True)
    cdf = image_histogram.cumsum() # cumulative distribution function
    cdf = 255 * cdf / cdf[-1] # normalize

    # use linear interpolation of cdf to find new pixel values
    image_equalized = np.interp(image.flatten(), bins[:-1], cdf)

    # return image_equalized.reshape(image.shape), cdf
    return image_equalized.reshape(image.shape)

def equalize(arr):
    arr = arr.astype('float')
    # usually do not touch the alpha channel
    # but here we do since it is yellow
    for i in range(4):
        # arr[...,i] = 255 * equalize_hist(arr[...,i])
        arr[...,i] = image_histogram_equalization(arr[...,i])                                  
    return arr

def normalize(arr, q=0.01):
    arr = arr.astype('float')
    # usually do not touch the alpha channel
    # but here we do since it is yellow
    # print('arr before',arr.shape,arr.min(),arr.mean(),arr.max())
    for i in range(arr.shape[-1]):
        # arr[...,i] = 255 * equalize_hist(arr[...,i])
        ai = arr[...,i]
        # print('ai ' + str(i) + ' before',i,ai.shape,ai.min(),ai.mean(),ai.max())
        qlow = np.percentile(ai,100*q)
        qhigh = np.percentile(ai,100*(1.0-q))
        if qlow == qhigh:
            arr[...,i] = 0.
        else:
            arr[...,i] = 255.*(np.clip(ai,qlow,qhigh) - qlow)/(qhigh - qlow)                              
        # print('ai ' + str(i) + ' after',i,ai.shape,ai.min(),ai.mean(),ai.max())
    # print('arr after',arr.shape,arr.min(),arr.mean(),arr.max())
    return arr


class MultiBandMultiLabelDataset(Dataset):
    
#     BANDS_NAMES = ['_red.png','_green.png','_blue.png','_yellow.png']
    BANDS_NAMES = ['_red.png','_green.png','_blue.png']
    
    def __len__(self):
        return len(self.images_df)
    
    def __init__(self, images_df, 
                 base_path, 
                 image_transform=None, 
                 augmentator=None,
                 train_mode=True    
                ):
        if not isinstance(base_path, pathlib.Path):
            base_path = pathlib.Path(base_path)
            
        self.images_df = images_df.reset_index()
        self.image_transform = image_transform
        self.augmentator = augmentator
        self.images_df.Id = self.images_df.Id.apply(lambda x: base_path / x)
        self.mlb = MultiLabelBinarizer(classes=list(LABEL_MAP.keys()))
        self.train_mode = train_mode
        self.cache = {}
                                 
    def __getitem__(self, index):
        # print('index class',index.__class__)
        if isinstance(index, torch.Tensor): index = index.item()
        if index in self.cache: 
            X, y = self.cache[index]
        else:
            y = None
            X = self._load_multiband_image(index)
            if self.train_mode:
                y = self._load_multilabel_target(index)
            self.cache[index] = (X,y)
        
        # augmentator can be for instance imgaug augmentation object
        if self.augmentator is not None:
            X = self.augmentator(X)
           
        if self.image_transform is not None:
            X = self.image_transform(X)
        
        return X, y 
        
    def _load_multiband_image(self, index):
        row = self.images_df.iloc[index]
        image_bands = []
        for band_name in self.BANDS_NAMES:
            p = str(row.Id.absolute()) + band_name
            pil_channel = PIL.Image.open(p)
            image_bands.append(pil_channel)
            
        # pretend its a RBGA image to support 4 channels
#         band4image = PIL.Image.merge('RGBA', bands=image_bands)
        band3image = PIL.Image.merge('RGB', bands=image_bands)

        # normalize each channel     
#         arr = np.array(band4image)
        arr = np.array(band3image)
#         # average red and yellow channels, orange
#         arr[...,0] = (arr[...,0] + arr[...,3])/2.0
#         arr = arr[...,:3]
        
        # arr = np.array(band3image)
        # print('arr shape',arr.shape)
        # if index==0: print(index,'hist before',histogram(arr))
        arr = normalize(arr)
        # if index==0: print(index,'hist after',histogram(arr))
        band3image = PIL.Image.fromarray(arr.astype('uint8'),'RGB')
#         band4image = PIL.Image.fromarray(arr.astype('uint8'),'RGBA')

        # histogram equalize each channel
        
#         arr = np.array(band4image)
#         # print('arr',arr.shape)
#         # if index==0: print(index,'hist before',histogram(arr))
#         arr = equalize(arr)
#         # if index==0: print(index,'hist after',histogram(arr))
#         band4image = PIL.Image.fromarray(arr.astype('uint8'),'RGBA')
        
#         return band4image
        return band3image

#         band3image = PIL.Image.new("RGB", band4image.size, (255, 255, 255))
#         band3image.paste(band4image, mask=band4image.split()[3]) 
#         band3image = band3image.resize((image_size,image_size), PIL.Image.ANTIALIAS)
#         return band3image
   
    
    def _load_multilabel_target(self, index):
        return list(map(int, self.images_df.iloc[index].Target.split(' ')))
    
        
    def collate_func(self, batch):
        labels = None
        images = [x[0] for x in batch]
        
        if self.train_mode:
            labels = [x[1] for x in batch]
            labels_one_hot  = self.mlb.fit_transform(labels)
            labels = torch.FloatTensor(labels_one_hot)
            
        
        # return torch.stack(images)[:,:4,:,:], labels
        return torch.stack(images), labels


In [13]:
class Batch(object):
    """Class encapsulating a batch before and after augmentation."""
    def __init__(self, identifiers, images, targets):
        self.identifiers = identifiers
        self.images = images
        self.images_aug = None
        self.targets = targets
        self.masks = None

class ImageLoader(object):
    """Class to load batches in the background."""

    def __init__(self, load_batch_func, nb_workers=1, queue_size=50, threaded=True):
        self.queue = multiprocessing.Queue(queue_size)
        self.workers = []
        for i in range(nb_workers):
            if threaded:
                worker = threading.Thread(target=self._load_batches, args=(load_batch_func, self.queue))
            else:
                worker = multiprocessing.Process(target=self._load_batches, args=(load_batch_func, self.queue))
            worker.daemon = True
            worker.start()
            self.workers.append(worker)

    def _load_batches(self, load_batch_func, queue):
        while True:
            queue.put(pickle.dumps(load_batch_func(), protocol=-1))

class BackgroundAugmenter(object):
    """Class to augment batches in the background (while training on
    the GPU)."""
    def __init__(self, augseq, queue_source, nb_workers, queue_size=50, threaded=False):
        assert 0 < queue_size <= 10000
        self.augseq = augseq
        self.queue_source = queue_source
        self.queue_result = multiprocessing.Queue(queue_size)
        self.workers = []
        for i in range(nb_workers):
            augseq.reseed()
            if threaded:
                worker = threading.Thread(target=self._augment_images_worker, args=(self.augseq, self.queue_source, self.queue_result))
            else:
                worker = multiprocessing.Process(target=self._augment_images_worker, args=(self.augseq, self.queue_source, self.queue_result))
            worker.daemon = True
            worker.start()
            self.workers.append(worker)

    def get_batch(self):
        """Returns a batch from the queue of augmented batches."""
        batch_str = self.queue_result.get()
        batch = pickle.loads(batch_str)
        return batch

    def _augment_images_worker(self, augseq, queue_source, queue_result):
        """Worker function that endlessly queries the source queue (input
        batches), augments batches in it and sends the result to the output
        queue."""
        while True:
            # wait for a new batch in the source queue and load it
            batch_str = queue_source.get()
            batch = pickle.loads(batch_str)

#             # augment the batch
#             if batch.images is not None and batch.masks is not None:
#                 augseq_det = augseq.to_deterministic()
#                 batch.images_aug = augseq_det.augment_images(batch.images)
#                 batch.masks_aug = augseq_det.augment_images(batch.masks)
#             elif batch.images is not None:
#                 batch.images_aug = augseq.augment_images(batch.images)
#             elif batch.masks is not None:
#                 batch.masks_aug = augseq.augment_images(batch.masks)

            if batch.images is not None:
                batch.images_aug = augseq.augment_images(batch.images)

            # send augmented batch to output queue
            queue_result.put(pickle.dumps(batch, protocol=-1))


In [14]:
# initialize image augmentation cascade
rarely = lambda aug: iaa.Sometimes(0.1, aug)
sometimes = lambda aug: iaa.Sometimes(0.25, aug)
often = lambda aug: iaa.Sometimes(0.75, aug)
seq = iaa.Sequential([
        iaa.Fliplr(0.5), # horizontally flip 50% of all images
        iaa.Flipud(0.5), # vertically flip 50% of all images
#         rarely(iaa.Superpixels(p_replace=(0, 1.0), n_segments=(20, 200))), # convert images into their superpixel representation
#         often(iaa.Crop(percent=(0, 0.1))), # crop images by 0-10% of their height/width
#         sometimes(iaa.GaussianBlur((0, 3.0))), # blur images with a sigma between 0 and 3.0
#         sometimes(iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5))), # sharpen images
#         sometimes(iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0))), # emboss images
#         # search either for all edges or for directed edges
#         rarely(iaa.Sometimes(0.5,
#             iaa.EdgeDetect(alpha=(0, 0.7)),
#             iaa.DirectedEdgeDetect(alpha=(0, 0.7), direction=(0.0, 1.0)),
#         )),
#         often(iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.2), per_channel=0.5)), # add gaussian noise to images
#         often(iaa.Dropout((0.0, 0.1), per_channel=0.5)), # randomly remove up to 10% of the pixels
#         # rarely(iaa.Invert(0.25, per_channel=True)), # invert color channels
#         often(iaa.Add((-10, 10), per_channel=0.5)), # change brightness of images (by -10 to 10 of original value)
#         often(iaa.Multiply((0.5, 1.5), per_channel=0.25)), # change brightness of images (50-150% of original value)
#         often(iaa.ContrastNormalization((0.5, 2.0), per_channel=0.5)), # improve or worsen the contrast
#         # sometimes(iaa.Grayscale(alpha=(0.0, 1.0))),
#         often(iaa.Affine(
#             scale={"x": (0.6, 1.4), "y": (0.6, 1.4)}, # scale images to 60-140% of their size, individually per axis
#             translate_percent={"x": (-0.3, 0.3), "y": (-0.3, 0.3)}, # translate by -30 to +30% percent (per axis)
#             rotate=(-45, 45), # rotate by -45 to +45 degrees
#             shear=(-16, 16), # shear by -16 to +16 degrees
#             order=[0, 1], # use any of scikit-image's interpolation methods
#             cval=(0, 255), # if mode is constant, use a cval between 0 and 255
#             mode=["constant", "edge"] # use any of scikit-image's warping modes (see 2nd image from the top for examples)
#         )),
#         sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)) # apply elastic transformations with random strengths
    ],
    random_order=True # do all of the above in random order
)

In [15]:
def parameters_string(module):
    lines = [
        "",
        "List of model parameters:",
        "=========================",
    ]

    row_format = "{name:<40} {shape:>20} ={total_size:>12,d}"
    params = list(module.named_parameters())
    for name, param in params:
        lines.append(row_format.format(
            name=name,
            shape=" * ".join(str(p) for p in param.size()),
            total_size=param.numel()
        ))
    lines.append("=" * 75)
    lines.append(row_format.format(
        name="all parameters",
        shape="sum of above",
        total_size=sum(int(param.numel()) for name, param in params)
    ))
    lines.append("")
    return "\n".join(lines)


In [16]:
composed_transforms_train = transforms.Compose([
    transforms.Resize(size=final_size),
    # transforms.RandomResizedCrop(size=224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(degrees=45),
    # tr.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    # tr.Normalize(mean=(0.485), std=(0.229)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.456]*4, std=[0.224]*4)
])

composed_transforms_test = transforms.Compose([
    transforms.Resize(size=final_size),
    # tr.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    # tr.Normalize(mean=(0.485), std=(0.229)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.456]*4, std=[0.224]*4)
])

In [17]:
#####################################
# model and main parameter settings #
#####################################

%run 'inceptionresnetv2_b8.ipynb'

device = "cuda"
# device = "cpu"

p = OrderedDict()  # Parameters to include in report
p['trainBatch'] = 24  # Training batch size
p['testBatch'] = 24  # Testing batch size

nEpochs = 24  # Number of epochs for training
resume_epoch = 0  # Default is 0, change if want to resume

p['lr'] = 3e-4  # Learning rate
p['step_size'] = 8
p['gamma'] = 0.1
p['wd'] = 1e-4  # Weight decay
p['momentum'] = 0.9  # Momentum
p['epoch_size'] = 15 # How many epochs to change learning rate
p['patience'] = 30 # epochs to wait for early stopping


In [18]:
num_classes = 28
gsize = 14
gpct = 95.
gstd = 0.1
gthresh = -2.0
eps = 1e-5

# save_dir_root = os.path.join(os.path.dirname(os.path.abspath(__file__)))
# exp_name = os.path.dirname(os.path.abspath(__file__)).split('/')[-1]

save_dir_root = './'

# save_dir = os.path.join(save_dir_root, 'run', 'run_' + str(run_id))
save_dir = save_dir_root + mname + '/'
os.makedirs(save_dir,exist_ok=True)
print(save_dir)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

gc.collect()

./iv3_b6/


21

In [ ]:
gc.collect()

for f in range(nfold):
    
    if f != fold: continue
    
    print('')
    print('*'*50)
    print(mname + ' fold ' + str(fold))
    print('*'*50)
    bname = mname+'/'+'best_'+str(fold)+'.pth'
    
    # Network definition
    net = InceptionResnetV2    pw = torch.tensor([10.]).float().to(device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pw)
(num_classes=28)
    
    print("Number of parameters:","{:,}".format(count_parameters(net)))
    # print(p.status())

    # classification loss
    # criterion = utils.cross_entropy2d
    # criterion = torch.nn.BCELoss()
    # criterion = dice_loss
    # criterion = BCELoss2d()
    # criterion = CombinedLoss(is_weight=False).cuda()
    # criterion = L.lovasz_hinge
    # criterion = L.lovasz2_bce1
    # criterion = L.lovasz_hinge
    # criterion = nn.BCEWithLogitsLoss()
    # criterion = FocalLoss()
    # criterion = ThreeWayLoss()

    # segmentation loss
    # criterion = L.lovasz_hinge
    # criterion = nn.CrossEntropyLoss()
    pw = torch.tensor([10.]).float().to(device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pw)
    # criterion = F.smooth_l1_loss

    if resume_epoch == 0:
        if len(mname0):
            bname0 = mname0+'/'+'best_'+str(fold)+'.pth'
            print(f'Initializing weights from {bname0}')
            # load best model
            best = torch.load(bname0, map_location='cpu')
            # print(best.keys())
            net.load_state_dict(best, strict=False)
    else:
        print(f'Initializing weights from {bname}')
        # load best model
        best = torch.load(bname, map_location='cpu')
        # print(best.keys())
        net.load_state_dict(best, strict=False)

    if gpu_id >= 0:
        print('Using GPU: {} '.format(gpu_id))
        torch.cuda.set_device(device=gpu_id)
        # net.cuda()
        net.train()
        net.to(device)
        
    gc.collect()
    
    # Logging into Tensorboard
    # log_dir = os.path.join(save_dir, 'models', datetime.now().strftime('%b%d_%H-%M-%S') + '_' + socket.gethostname())
    log_dir = os.path.join('tensorboard', mname + '_' + str(fold))
    writer = SummaryWriter(log_dir=log_dir)

    # Use the following optimizer
    optimizer = torch.optim.Adam(net.parameters(), lr=p['lr'])
#     optimizer = optim.SGD(net.parameters(), lr=p['lr'], momentum=p['momentum'],
#                           weight_decay=p['wd'])
#     optimizer = torch.optim.Adadelta(net.parameters(), lr=1.0, rho=0.9, eps=1e-06,
#                                      weight_decay=1e-6)
    p['optimizer'] = str(optimizer)

#     scheduler = LambdaLR(optimizer, lr_lambda=cyclic_lr)
#     scheduler.base_lrs = list(map(lambda group: 1.0, optimizer.param_groups))  
#     scheduler = ReduceLROnPlateau(optimizer, factor=0.2, patience=5, verbose=True, 
#                                   threshold=0.0, threshold_mode='abs')

    scheduler = StepLR(optimizer, step_size=p['step_size'], gamma=p['gamma'])

    torch.cuda.empty_cache()

    file_list_val = [f for f in file_list if folds[f]==fold]
    file_list_train = [f for f in file_list if f not in file_list_val]
    print('Training on ' + str(len(file_list_train)) + \
          ' and validating on ' + str(len(file_list_val)))
    
 
    db_train = MultiBandMultiLabelDataset(df.loc[file_list_train], 
                                          base_path=PATH_TO_TRAIN,
                                          image_transform=composed_transforms_train)
    db_val = MultiBandMultiLabelDataset(df.loc[file_list_val], 
                                        base_path=PATH_TO_TRAIN,
                                        image_transform=composed_transforms_test)

    # construct sampling weights as max of reciprocal class frequencies
    ylist = [t.split(' ') for t in db_train.images_df.Target]
    # print(ylist[:5])
   
    # build one-hot matrix
    y = np.zeros((db_train.images_df.shape[0],28))
    for i,l in enumerate(ylist):
        for j in range(len(l)): y[i,int(l[j])] = 1.
    # print(y[:20])

    # sampling weights 
    w = 1.0/np.mean(y,axis=0)
    # w = np.clip(w, 0., 1000.)
    np.set_printoptions(precision=4,linewidth=80,suppress=True)
    print('Sampling weights:')
    print(w)

    # replace 1s with weights in the one-hot matrix
    for i,l in enumerate(ylist):
        for j in range(len(l)): y[i,int(l[j])] = w[int(l[j])]
    # print(y[:10])

    # use maximum weight when there are multiple targets
    samples_weight = np.amax(y,axis=1)
    samples_weight = torch.from_numpy(samples_weight)
    sampler = WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'),
                                    len(samples_weight))

    trainloader = DataLoader(db_train, collate_fn=db_train.collate_func,
                             batch_size=p['trainBatch'], sampler=sampler,
                             num_workers=14)
    valloader = DataLoader(db_val, collate_fn=db_train.collate_func,
                           batch_size=p['testBatch'], shuffle=False,
                          num_workers=14)
    
#     # function to generate batches within ImageLoader with no arguments
#     def load_training_batch():
#         examples_batch = random.sample(list(db_train.images_df.Id.values), p['trainBatch'])
#         blist = [db_train[ex] for ex in examples_batch]
#         images = [b[0] for b in blist]
#         targets = [b[1] for b in blist]
#         return Batch(identifiers=None, images=images, targets=targets)

#     img_loader = ImageLoader(load_training_batch, nb_workers=6)
#     bg_augmenter = BackgroundAugmenter(seq, img_loader.queue, nb_workers=8)

    utils.generate_param_report(os.path.join(save_dir, mname + '.txt'), p)

    # number of batches
    num_img_tr = len(trainloader)
    num_img_ts = len(valloader)
    
    print('Image size:', final_size)
    print('Batch size:', p['trainBatch'])
    print('Batches per epoch:', num_img_tr)
    print('Epochs:', nEpochs)
    print('Loss:', criterion)
    # print('Learning rate: ', p['lr'])
    print('')
   
    running_loss_tr = 0.0
    running_loss_ts = 0.0
    aveGrad = 0
    bname = mname+'/'+'best_'+str(fold)+'.pth'
    # print("Training Network")
    history = {}
    history['epoch'] = []
    history['train'] = []
    history['val'] = []
    history['delta'] = []
    history['f1'] = []
    history['time'] = []
    best_val = -999
    bad_epochs = 0
    start_time = timeit.default_timer()
    total_time = 0
    prev_lr = 999

    # Main Training and Testing Loop
    for epoch in range(resume_epoch, nEpochs):

#         if (epoch > 0) and (epoch % p['epoch_size'] == 0):
#             lr_ = utils.lr_poly(p['lr'], epoch, nEpochs, 0.9)
#             print('(poly lr policy) learning rate', lr_)
#             print('')
#             optimizer = optim.SGD(net.parameters(), lr=lr_, momentum=p['momentum'],
#                                   weight_decay=p['wd'])

        scheduler.step()
        lr = optimizer.param_groups[0]['lr']
        if lr != prev_lr: 
            print('learning rate = %.6f' % lr)
            prev_lr = lr

        net.train()
        
        train_loss = []
        ns = 0
        
        # for ii in range(num_img_tr):
        for ii, sample_batched in enumerate(trainloader):
            
            inputs, gts = sample_batched[0], sample_batched[1]
#             inp = inputs.numpy()
#             print('image stats', inp.shape, inp.min(), inp.mean(), inp.max())
#             for j in range(4):
#                 inpj = inp[:,j]
#                 print('image ' + str(j) + ' stats', inpj.shape, inpj.min(), inpj.mean(),
#                       inpj.max())
            
            # use thresholded green channel as ground truth mask for current classes
            gi = inputs.numpy()[:,1].copy()
#             print('gi stats', gi.shape, gi.min(), gi.mean(), gi.max())
            bsize = gi.shape[0]
            gmask = np.zeros((bsize, num_classes, gsize, gsize)).astype(int)
            for jj in range(bsize):
                gij = gi[jj]
#                 print('gij before filter', gij.shape, gij.min(), gij.mean(), gij.max())
#                 gij = gaussian_filter(gij,gstd)
#                 print('gij after filter', gij.shape, gij.min(), gij.mean(), gij.max())
                gij = (gij > gthresh).astype(float)
#                 print('gij after thresh', gij.shape, gij.min(), gij.mean(), gij.max())
                gr = cv2.resize(gij, (gsize,gsize), interpolation=interp)
#                 print('gr before dilation', gr.shape, gr.min(), gr.mean(), gr.max())
                gr = binary_dilation(gr).astype(int)
#                 print('gr after dilation', gr.shape, gr.min(), gr.mean(), gr.max())
#                 gin = gi[jj]
#                 gin = (gin - gin.min())/(gin.max()-gin.min()+1e-6)
#                 grn = cv2.resize(gin, (gsize,gsize), interpolation=interp)
#                 print('grn stats', grn.shape, grn.min(), grn.mean(), grn.max())
#                 gr = (gr > gthresh).astype(bool).astype(int)
#                 print('gr mean batch', jj, np.mean(gr))
                for kk in np.nonzero(gts[jj]):
                    gmask[jj,kk] = gr
#                 print(batch, 'y', gts[jj])
#                 print(batch, 'gmask mean', np.average(gmask[jj], axis=(1,2)))
            
            gmask = torch.from_numpy(gmask).float()
            
            # keep track of sampling proportions
            gt = gts.cpu().detach().numpy()
            gs = np.sum(gt,axis=0)
            if ii==0: gtsum = gs
            else: gtsum += gs
            ns += bsize

            inputs = inputs.type(torch.float).to(device)
            gts = gts.to(device)
            gmask = gmask.to(device)
            
            # predictions are heat maps on a probability scale
            logits = net(inputs)
            
#             class_loss = criterion(logits, gts)
            
#             first = True
#             for kk in range(num_classes):
#                 lossk = criterion2(seg[:,kk], gmask[:,kk])
#                 # print('seg_loss batch', jj, ' class', kk, lossjk.item())
#                 if first: 
#                     seg_loss = lossk
#                     first = False
#                 else: seg_loss = seg_loss + lossk
#             seg_loss = seg_loss / num_classes

             
            # print('class_loss', class_loss.item())
            # print('seg_loss', seg_loss.item())
#             loss = class_loss + 0.5 * seg_loss

            loss = criterion(logits, gmask)
#             print(ii, loss.item())
            
            optimizer.zero_grad()
            loss.backward()
            
            # adamw
            for group in optimizer.param_groups:
                for param in group['params']:
                    param.data = param.data.add(-p['wd'] * group['lr'], param.data)

            optimizer.step()
            train_loss.append(loss.item())                        
            running_loss_tr += loss.item()

        
        print('epoch ' + str(epoch) + ' training class proportions:')
        print(gtsum/ns)
        
        # validation
        net.eval()
        
        with torch.no_grad():

            val_loss = []
            val_predictions = []
            val_targets = []
        
            for ii, sample_batched in enumerate(valloader):

                # inputs, gts = sample_batched['image'], sample_batched['gt']
                inputs, gts = sample_batched[0], sample_batched[1]

                # use thresholded green channel as ground truth mask for current classes
                gi = inputs.numpy()[:,1].copy()
                bsize = gi.shape[0]
                gmask = np.zeros((bsize, num_classes, gsize, gsize)).astype(float)
                for jj in range(bsize):
                    gr = cv2.resize(gi[jj], (gsize,gsize), interpolation=interp)
                    # print('gr shape', gr.shape)
                    # gr = (gr > gthresh).astype(bool).astype(int)
                    # print('gr mean batch', jj, np.mean(gr))
                    for kk in np.nonzero(gts[jj]):
                        gmask[jj,kk] = gr

                gmask = torch.from_numpy(gmask).float()
                
                # tta horizontal flip
                inputs2 = inputs.numpy()[:,:,:,::-1].copy()
                inputs2 = torch.from_numpy(inputs2)

                inputs = inputs.type(torch.float).to(device)
                inputs2 = inputs2.type(torch.float).to(device)

                # predictions are on a logit scale
                logits = net(inputs)
                logits2 = net(inputs2)
                logits2 = logits2.cpu().detach().numpy()[:,:,:,::-1].copy()
                logits2 = torch.from_numpy(logits2).to(device)
                logits = (logits + logits2)/2.0
                
                loss = criterion(logits, gmask.to(device))

                running_loss_ts += loss.item()
                val_loss.append(loss.item())

                # save results to compute F1 on validation set
                preds = logits.cpu().detach().numpy()
                gt = gts.cpu().detach().numpy()

                val_predictions.append(preds)
                val_targets.append(gt)

            vps = np.vstack(val_predictions)
            vts = np.vstack(val_targets)
            
            # competition metric
            # use percentile to as single prediction for f1
            vpsp = np.percentile(vps, gpct, axis=(2,3))
            thresholds = np.linspace(-5, 5, 101)
            scores = np.array([f1_score(vts, np.int32(vpsp > t),
                                    average='macro') for t in thresholds])
            threshold_best_index = np.argmax(scores)
            vf1 = scores[threshold_best_index]
            tbest = thresholds[threshold_best_index]
            # vf1 = f1_score(vts,(vps > 0).astype(int), average='macro')

            if vf1 > best_val:
                star = '*'
                best_val = vf1
                torch.save(net.state_dict(), bname)
                bad_epochs = 0
            else:
                star = ' '
                bad_epochs += 1
                
            # print progress
            # running_loss_ts = running_loss_ts / num_img_ts

            tl = np.mean(train_loss)
            vl = np.mean(val_loss)

            stop_time = timeit.default_timer()
            diff_time = stop_time - start_time
            total_time += diff_time/60.
            start_time = timeit.default_timer()

            print('epoch %d  train %6.4f  val %6.4f  delta %6.4f  f1 %6.4f%s  thresh %3.1f  time %2.0f%s\n' % \
                  (epoch, tl, vl, vl-tl, vf1, star, tbest, diff_time, 's'))
            writer.add_scalar('loss', tl, epoch)
            writer.add_scalar('val_loss', vl, epoch)
            writer.add_scalar('delta', vl-tl, epoch)
            writer.add_scalar('val_f1', vf1, epoch)
            writer.add_scalar('thresh', tbest, epoch)
            writer.add_scalar('time', diff_time, epoch)
            # print('Running Loss: %f\n' % running_loss_ts)
            # print('Mean Loss: %f\n' % np.mean(val_loss))
            running_loss_tr = 0
            running_loss_ts = 0

            history['epoch'].append(epoch)
            history['train'].append(tl)
            history['val'].append(vl)
            history['f1'].append(vf1)
            history['time'].append(diff_time)

            if bad_epochs > p['patience']:
                print('early stopping, best validation loss %6.4f, total time %4.1f minutes \n' % \
                      (best_val, total_time))
                break
            
    writer.close()

    # plot history
    fig, (ax_loss) = plt.subplots(1, 1, figsize=(8,4))
    ax_loss.plot(history['epoch'], history['train'], label="Train loss")
    ax_loss.plot(history['epoch'], history['val'], label="Validation loss")
    plt.show()
    plt.gcf().clear()


**************************************************
iv3_b6 fold 2
**************************************************
Number of parameters: 21,852,066
Initializing weights from iv3_b5/best_2.pth
Using GPU: 0 
Training on 19368 and validating on 11704
Sampling weights:
[   2.3482   31.1383    8.8519   23.6195   18.641    12.9552   32.0662
   11.6255  807.      922.2857 1760.7273   31.803    47.1241   69.6691
   29.1687 1291.2      56.4665  110.0455   33.5667   23.7062  312.3871
    7.0971   37.5349   10.802   123.3631    3.4499   94.478  3873.6   ]
Image size: 512
Batch size: 24
Batches per epoch: 807
Epochs: 24
Loss: BCEWithLogitsLoss()

learning rate = 0.000300
epoch 0 training class proportions:
[0.3134 0.0415 0.102  0.0501 0.0499 0.0755 0.0553 0.0568 0.0428 0.0623 0.0425
 0.0425 0.0423 0.0425 0.0542 0.0427 0.0698 0.0431 0.0513 0.0442 0.0403 0.0926
 0.0429 0.0579 0.0408 0.1834 0.0394 0.0374]
epoch 0  train 1.5622  val 0.9092  delta -0.6530  f1 0.3982*  thresh 3.1  time 833s

epoch 1 t

In [ ]:
# load best model
best = torch.load(bname, map_location='cpu')
# print(best.keys())
net.load_state_dict(best)
net = net.eval()

In [ ]:
with torch.no_grad():
    # predict validation set
    val_logits = []
    val_y = []
    # for image, mask in tqdm.tqdm(data.DataLoader(dataset_val, batch_size = 30)):
    batch = 0
    for image, y in valloader:
        # test-time augmentation with horizontal flipping
        image2 = image.numpy()[:,:,:,::-1].copy()
        image2 = torch.from_numpy(image2)
        image = image.type(torch.float).to(device)
        image2 = image2.type(torch.float).to(device)
        logits = net(image)
        logits = logits.cpu().detach().numpy()
        logits2 = net(image2)
        logits2 = logits2.cpu().detach().numpy()
        logits2 = logits2[:,:,:,::-1]
        logits = (logits + logits2)/2.0
        val_logits.append(logits)

        y = y.cpu().detach().numpy()
        val_y.append(y)

        batch += 1

    vls = np.vstack(val_logits)
    vys = np.vstack(val_y)

    print(vls.shape, vys.shape)

In [ ]:
print(logits.shape,logits.min(),logits.mean(),logits.max())
print(logits2.shape,logits2.min(),logits2.mean(),logits2.max())

In [ ]:
clip = 15
vpc = np.array([np.clip(logits.flatten(),0.,clip), np.clip(logits2.flatten(),0.,clip)])

# tpsf = np.hstack([c.reshape((-1,1)) for c in tps])
print(vpc.shape)

In [ ]:
np.corrcoef(vpc)

In [ ]:
# save out-of-fold predictions
oof_ids = file_list_val
poof = vls.copy()
yoof = vys.copy()

oof = [oof_ids, poof, yoof]
fname = 'oof/'+mname+'_'+str(fold)+'.pkl'
pickle.dump(oof,open(fname,'wb'))
print(fname)

In [ ]:
# grid search for best threshold
# note predictions and thresholds are on logit scale
vlsp = np.percentile(vls, gpct, axis=(2,3))
# vlsp = np.average(vls, axis=(2,3))

thresholds = np.linspace(-5, 10, 151)
scores = np.array([f1_score(vys, (vlsp > t).astype(int), average='macro') \
                 for t in thresholds])

threshold_best_index = np.argmax(scores)
score_best = scores[threshold_best_index]
threshold_best = thresholds[threshold_best_index]
print('')
print('f1_best',score_best)
print('threshold_best',threshold_best)
print('')

plt.plot(thresholds, scores)
plt.plot(threshold_best, score_best, "xr", label="Best threshold")
plt.xlabel("Threshold")
plt.ylabel("F1")
plt.title("Threshold vs F1 ({}, {})".format(threshold_best, score_best))
plt.legend()
plt.show()
plt.gcf().clear()

In [ ]:
vf = vlsp.flatten()
print(vf.min(),vf.mean(),vf.max(),vf.shape)
sns.distplot(vf)
plt.title("Distribution of Predictions (Logit Scale) for Fold " + str(fold+1))
plt.show()
plt.gcf().clear()

In [ ]:
np.mean(vys,axis=0)

In [ ]:
# error analysis
from sklearn.metrics import confusion_matrix

cm = [confusion_matrix(vys[:,i], (vlsp[:,i] > threshold_best).astype(int)) \
       for i in range(vys.shape[1])]
fm = [f1_score(vys[:,i], (vlsp[:,i] > threshold_best).astype(int)) \
       for i in range(vys.shape[1])]
for i in range(vys.shape[1]):
    print(LABEL_MAP[i])
    print(cm[i], '%4.2f' % fm[i])
    print('')

In [ ]:
np.mean(fm)

In [ ]:
# fm1 = [f for f in fm if f > 0]
# print(len(fm1))
# print(np.mean(fm1))

In [ ]:
f1b = np.array([f1_score(y, (l > threshold_best).astype(int)) \
                 for y,l in zip(vys,vlsp)])
print(f1b.min(),f1b.mean(),f1b.max())
sns.distplot(f1b)
plt.title("Distribution of Sample F1 Scores for Fold " + str(fold))
plt.show()
plt.gcf().clear()

In [ ]:
len(f1b)

In [ ]:
# plot validation images with scores
# sort from worst to best
order = f1b.argsort()
max_images = 90
# max_images = len(file_list_val)
start = 0
# start = 200
grid_width = 10
grid_height = int(max_images / grid_width)
# print(max_images,grid_height,grid_width)

file_list_val_reordered = [file_list_val[order[i]] for i,f in enumerate(file_list_val)]

for i, idx in enumerate([file_list_val_reordered[i] for i in range(start,(start+max_images))]):
    imod = i % 30
    if imod == 0:
        fig, axs = plt.subplots(3, 10, figsize=(30, 10))
    img, y = db_val[order[i]]

    img = img.data.numpy()[1]
    img = img[y_min_pad:(image_size - y_max_pad), x_min_pad:(image_size - x_max_pad)]

    true = np.nonzero(vys[order][start+i])
    true_str = ' '.join(map(str, true))
    pred = np.nonzero((vlsp[order][start+i] > threshold_best).astype(int))
    pred_str = ' '.join(map(str, pred))
    ax = axs[int(imod / grid_width), imod % grid_width]
    ax.imshow(img, cmap='Greens')
    ax.set_title(str(i) + '   ' + idx[:13] + '\n' + true_str + '  ' + pred_str)
    # ax.set_xlabel(str(round(ioub[i], 3)))
    ax.set_xlabel('%4.2f' % (f1b[order][start+i]))
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    if imod == 29:
        # plt.suptitle("Green: salt, Red: prediction. Top-left: coverage class, Top-right: salt coverage, Bottom-left: depth, Bottom-right: IOU")
        plt.show()
        plt.gcf().clear()
        gc.collect()    

In [ ]:
print(ss.head())
print(ss.shape)

In [ ]:
db_test = MultiBandMultiLabelDataset(ss, train_mode=False,
                                    base_path=PATH_TO_TEST,
                                    image_transform=composed_transforms_test)

In [ ]:
testloader = DataLoader(db_test, collate_fn=db_test.collate_func,
                       batch_size=p['testBatch'], shuffle=False,
                      num_workers=14)

In [ ]:
clip = 20
with torch.no_grad():
    print('predicting test set for bagging')
    tp = {}
    for i in range(8): tp[i] = []
    
    # 8-way TTA
    # for image in tqdm.tqdm(data.DataLoader(test_dataset, batch_size = 30)):
    for image in testloader:
        i = 0
        image1 = image[0].numpy().copy()
        # move channels last for augmentation
        image1 = np.transpose(image1, (0, 2, 3, 1))
       
        image = image[0].type(torch.float).to(device)
        logits = net(image)
        logits = logits.cpu().detach().numpy()
        logits = np.clip(logits,-clip,clip)
        tp[i].append(logits)
        i += 1
       
        for degrees in [90, 180, 270]:
            IAA = iaa.Affine(rotate=degrees)
            image2 = np.array([IAA.augment_image(imi) for imi in image1])
            # move channels first for pytorch
            image2 = np.transpose(image2, (0, 3, 1, 2))
            image2 = torch.from_numpy(image2)
            image2 = image2.type(torch.float).to(device)
            logits2 = net(image2)
            logits2 = logits2.cpu().detach().numpy()
            logits2 = np.clip(logits2,-clip,clip)
            IAA = iaa.Affine(rotate=-degrees)
            logits2 = np.transpose(logits2, (0, 2, 3, 1))
            logits2 = np.array([IAA.augment_image(imi) for imi in logits2])
            logits2 = np.transpose(logits2, (0, 3, 1, 2))
            tp[i].append(logits2)
            i += 1
        
        # horizontally flip image1
        IAA = iaa.Fliplr(1.0)
        image1 = np.array([IAA.augment_image(imi) for imi in image1])
        image2 = np.transpose(image1, (0, 3, 1, 2))
        image2 = torch.from_numpy(image2)
        image2 = image2.type(torch.float).to(device)
        logits2 = net(image2)
        logits2 = logits2.cpu().detach().numpy()
        logits2 = np.clip(logits2,-clip,clip)
        logits2 = np.transpose(logits2, (0, 2, 3, 1))
        logits2 = np.array([IAA.augment_image(imi) for imi in logits2])
        logits2 = np.transpose(logits2, (0, 3, 1, 2))
        tp[i].append(logits2)
        i += 1
        
        # rotations again on flipped image
        for degrees in [90, 180, 270]:
            IAA = iaa.Affine(rotate=degrees)
            image2 = np.array([IAA.augment_image(imi) for imi in image1])
            image2 = np.transpose(image2, (0, 3, 1, 2))
            image2 = torch.from_numpy(image2)
            image2 = image2.type(torch.float).to(device)
            logits2 = net(image2)
            logits2 = logits2.cpu().detach().numpy()
            logits2 = np.clip(logits2,-clip,clip)
            IAA = iaa.Affine(rotate=-degrees)
            logits2 = np.transpose(logits2, (0, 2, 3, 1))
            logits2 = np.array([IAA.augment_image(imi) for imi in logits2])
            logits2 = np.transpose(logits2, (0, 3, 1, 2))
            tp[i].append(logits2)
            i += 1
         
    tps = np.array([np.vstack(tp[i]) for i in range(8)])
    print(tps.shape)

tpsf = np.hstack([c.reshape((-1,1)) for c in tps])
print(tpsf.shape)

np.set_printoptions(precision=3,linewidth=100)
print(np.corrcoef(tpsf, rowvar=False))

ptest = np.median(tps,axis=0)
ptesta = np.amax(tps,axis=0)
print(ptest.shape)

In [ ]:
# show some test images
nshow = 50
start = np.random.randint(len(test_file_list)-nshow)
stop = start + nshow
grid_width = 10
grid_height = int(max_images / grid_width)
# print(max_images,grid_height,grid_width)
ni = 10
for j in range(int(start/10),int(stop/10)):
    jj = j*10
    fig, axs = plt.subplots(3, ni, figsize=(20,8))
    for i in range(ni):
        img = db_test[jj+i]
        img = img[0].data.numpy()
        img = img[:,y_min_pad:(image_size - y_max_pad),
                    x_min_pad:(image_size - x_max_pad)]
        # img = cv2.resize(img,(ori_size,ori_size),interpolation=interp)
        pred = np.nonzero((ptest[jj+i] > threshold_best).astype(int))
        # pred_str = list(pred)
        # pred_str = np.char.mod('%d', pred)
        # pred_str = " ".join(pred_str)
        pred_str = ' '.join(map(str, pred))
        axs[0][i].imshow(img[0], cmap="Reds")
        axs[1][i].imshow(img[1], cmap="Greens")
        axs[2][i].imshow(img[2], cmap="Blues")
#         axs[3][i].imshow(img[3], cmap="Oranges")
        axs[0][i].set_title(pred_str)
    # fig.suptitle("Top row: original, bottom row: green channel")
    plt.show()
    plt.gcf().clear()

#     # clean up to save on memory accumulation across folds
#     del net
#     del inputs, gts
#     del image, image2
#     del writer, scheduler, optimizer
#     del y_pred, y_pred2
#     torch.cuda.empty_cache()
#     gc.collect()

In [ ]:
sub = [test_file_list, ptest, ptesta]
fname = 'sub/'+mname+'_'+str(fold)+'_mm.pkl'
pickle.dump(sub,open(fname,'wb'))
print(fname)